## XGBoost- restricted

In [1]:
import pandas as pd
import numpy as np
from sklearn import cluster, covariance, manifold
import numpy as np
from sklearn.metrics import r2_score
import math
import matplotlib.pyplot as plt
import time
import os

In [2]:
from xgboost import XGBRegressor
from scipy.optimize import minimize
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.optimize import minimize
from sklearn.feature_selection import RFECV

In [3]:
import optuna

In [4]:
# import warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import root_mean_squared_error

In [6]:
path = r'C:\Users\User\Documents\Atividades_andamento\Nayana\corrige_codigos'
os.chdir(path)

In [7]:
#df =pd.read_excel('custos_versao_jun_17_2023.xlsx')
df =pd.read_excel("custos_versao_mar_14B_2024.xlsx")
df = df.drop(['Local','Barragens'],axis=1)
names = df.columns

In [8]:
df.shape

(33, 78)

In [9]:
word ='Custo_barragem' 
word = 'Dams'
X = df.drop([word],axis=1)
y = df[word]
y = np.log10(y)

escalor =  StandardScaler() # escalor = MinMaxScaler()
y_norm = escalor.fit_transform(y.values.reshape(-1,1))

## XGBoost



In [10]:
restricted =['ReinfEmbank','GroundwaQMon',
             'ErosProcMain','Volume','Area','Height']

In [11]:
X_restricted = df[restricted]

x_scaler = StandardScaler() # x_scaler = MinMaxScaler()
y_scaler = StandardScaler()# y_scaler = MinMaxScaler()

X_norm = x_scaler.fit_transform(X_restricted)
y_norm = y_scaler.fit_transform(y.values.reshape(-1,1))

In [12]:
X,y = X_norm,y_norm

In [13]:
def mine_test(set_parameters, X=X, y=y): # testa o ajuste
  rows=X.shape[0]
  cols =X.shape[1]

  y_pred   = []

  for i in range(rows):
    X_train = np.delete(X,i,axis=0)
    y_train = np.delete(y,i,axis=0)
    X_test  = X[i,:].reshape(1,cols)
    y_test  = y[i]
    model =  XGBRegressor(**set_parameters)
    model = model.fit(X_train, y_train.reshape(-1,1))
    yi = model.predict(X_test)
    y_pred.append(yi)

  y_pred = np.array(y_pred)

  resp = mean_absolute_error(y, y_pred)
  

  return -resp


In [14]:
# Define the objective function and suggest hyperparameters values
def objective(trial):
    learning_rate = trial.suggest_float('learning_rate', 0, 1)
    max_depth = trial.suggest_int('max_depth', 0, 20)
    min_child_weight= trial.suggest_int('min_child_weight', 0, 20)
    n_estimators= trial.suggest_int('n_estimators', 2, 300)
    gamma= trial.suggest_int('gamma', 0, 20)
    lambda_ = trial.suggest_int('lambda', 0, 20)
    
    set_parameters = {'booster': 'gbtree',
                      'verbosity':0,
                      'learning_rate': learning_rate,
                      'gamma': gamma, 
                      'lambda': lambda_,
                      'min_child_weight': min_child_weight,
                      'max_depth': max_depth, 
                      'n_estimators': n_estimators}
    adjust = mine_test( set_parameters)
    
    
    return adjust

In [15]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

[I 2024-08-13 17:22:47,591] A new study created in memory with name: no-name-d0661ad8-701b-42dc-9d7f-80aba9c415ef
[I 2024-08-13 17:22:47,947] Trial 0 finished with value: -0.6184758446453222 and parameters: {'learning_rate': 0.953347474374145, 'max_depth': 1, 'min_child_weight': 15, 'n_estimators': 13, 'gamma': 13, 'lambda': 1}. Best is trial 0 with value: -0.6184758446453222.
[I 2024-08-13 17:22:48,898] Trial 1 finished with value: -0.6265318583427999 and parameters: {'learning_rate': 0.5754370015941113, 'max_depth': 13, 'min_child_weight': 8, 'n_estimators': 116, 'gamma': 6, 'lambda': 9}. Best is trial 0 with value: -0.6184758446453222.
[I 2024-08-13 17:22:49,195] Trial 2 finished with value: -0.6797242955525186 and parameters: {'learning_rate': 0.7409838905823215, 'max_depth': 10, 'min_child_weight': 12, 'n_estimators': 10, 'gamma': 9, 'lambda': 11}. Best is trial 0 with value: -0.6184758446453222.
[I 2024-08-13 17:22:50,067] Trial 3 finished with value: -0.8521738100187027 and para

[I 2024-08-13 17:23:34,450] Trial 31 finished with value: -0.5699517735977342 and parameters: {'learning_rate': 0.5200240804743075, 'max_depth': 15, 'min_child_weight': 4, 'n_estimators': 229, 'gamma': 2, 'lambda': 11}. Best is trial 28 with value: -0.48775466210662055.
[I 2024-08-13 17:23:36,362] Trial 32 finished with value: -0.5781200792463688 and parameters: {'learning_rate': 0.5918025068895434, 'max_depth': 17, 'min_child_weight': 2, 'n_estimators': 261, 'gamma': 4, 'lambda': 9}. Best is trial 28 with value: -0.48775466210662055.
[I 2024-08-13 17:23:38,493] Trial 33 finished with value: -0.6501449642377413 and parameters: {'learning_rate': 0.4593501212844992, 'max_depth': 14, 'min_child_weight': 0, 'n_estimators': 280, 'gamma': 6, 'lambda': 11}. Best is trial 28 with value: -0.48775466210662055.
[I 2024-08-13 17:23:39,562] Trial 34 finished with value: -0.4708530851383074 and parameters: {'learning_rate': 0.9059290524980406, 'max_depth': 13, 'min_child_weight': 4, 'n_estimators': 

[I 2024-08-13 17:24:34,774] Trial 62 finished with value: -0.3617566112348607 and parameters: {'learning_rate': 0.2911277970745683, 'max_depth': 13, 'min_child_weight': 1, 'n_estimators': 148, 'gamma': 0, 'lambda': 4}. Best is trial 52 with value: -0.34873475430158607.
[I 2024-08-13 17:24:36,519] Trial 63 finished with value: -0.44325434436664235 and parameters: {'learning_rate': 0.28943086813772567, 'max_depth': 10, 'min_child_weight': 0, 'n_estimators': 145, 'gamma': 1, 'lambda': 4}. Best is trial 52 with value: -0.34873475430158607.
[I 2024-08-13 17:24:38,049] Trial 64 finished with value: -0.584194599495936 and parameters: {'learning_rate': 0.11766497454927217, 'max_depth': 16, 'min_child_weight': 2, 'n_estimators': 193, 'gamma': 3, 'lambda': 7}. Best is trial 52 with value: -0.34873475430158607.
[I 2024-08-13 17:24:39,099] Trial 65 finished with value: -0.5077457824089519 and parameters: {'learning_rate': 0.27284417953545437, 'max_depth': 18, 'min_child_weight': 3, 'n_estimators':

[I 2024-08-13 17:25:43,201] Trial 93 finished with value: -0.5205689626762979 and parameters: {'learning_rate': 0.2777274625595907, 'max_depth': 15, 'min_child_weight': 0, 'n_estimators': 238, 'gamma': 1, 'lambda': 11}. Best is trial 78 with value: -0.3070491569047886.
[I 2024-08-13 17:25:48,287] Trial 94 finished with value: -0.3158752407708461 and parameters: {'learning_rate': 0.3538441872705733, 'max_depth': 15, 'min_child_weight': 2, 'n_estimators': 235, 'gamma': 0, 'lambda': 10}. Best is trial 78 with value: -0.3070491569047886.
[I 2024-08-13 17:25:50,354] Trial 95 finished with value: -0.8739523121110325 and parameters: {'learning_rate': 0.3051804788582065, 'max_depth': 16, 'min_child_weight': 2, 'n_estimators': 259, 'gamma': 18, 'lambda': 11}. Best is trial 78 with value: -0.3070491569047886.
[I 2024-08-13 17:25:52,292] Trial 96 finished with value: -0.4984704317154296 and parameters: {'learning_rate': 0.357869993364152, 'max_depth': 16, 'min_child_weight': 3, 'n_estimators': 24

[I 2024-08-13 17:27:15,443] Trial 124 finished with value: -0.33890564290753356 and parameters: {'learning_rate': 0.543546570288204, 'max_depth': 18, 'min_child_weight': 1, 'n_estimators': 257, 'gamma': 0, 'lambda': 11}. Best is trial 115 with value: -0.2984978041375866.
[I 2024-08-13 17:27:18,938] Trial 125 finished with value: -0.34533889286674774 and parameters: {'learning_rate': 0.5465590344686133, 'max_depth': 18, 'min_child_weight': 1, 'n_estimators': 264, 'gamma': 0, 'lambda': 11}. Best is trial 115 with value: -0.2984978041375866.
[I 2024-08-13 17:27:20,792] Trial 126 finished with value: -0.5490274720326678 and parameters: {'learning_rate': 0.5877189944980995, 'max_depth': 19, 'min_child_weight': 2, 'n_estimators': 245, 'gamma': 1, 'lambda': 11}. Best is trial 115 with value: -0.2984978041375866.
[I 2024-08-13 17:27:22,957] Trial 127 finished with value: -0.7544399022448287 and parameters: {'learning_rate': 0.4807318837696125, 'max_depth': 17, 'min_child_weight': 3, 'n_estimat

[I 2024-08-13 17:28:58,807] Trial 155 finished with value: -0.5253137585298095 and parameters: {'learning_rate': 0.70283862501721, 'max_depth': 20, 'min_child_weight': 9, 'n_estimators': 284, 'gamma': 0, 'lambda': 11}. Best is trial 115 with value: -0.2984978041375866.
[I 2024-08-13 17:29:01,169] Trial 156 finished with value: -0.47451892879140023 and parameters: {'learning_rate': 0.6651271192739956, 'max_depth': 19, 'min_child_weight': 4, 'n_estimators': 299, 'gamma': 1, 'lambda': 8}. Best is trial 115 with value: -0.2984978041375866.
[I 2024-08-13 17:29:05,978] Trial 157 finished with value: -0.3317762410087279 and parameters: {'learning_rate': 0.7176447427526417, 'max_depth': 18, 'min_child_weight': 1, 'n_estimators': 262, 'gamma': 0, 'lambda': 12}. Best is trial 115 with value: -0.2984978041375866.
[I 2024-08-13 17:29:09,677] Trial 158 finished with value: -0.3345036406402472 and parameters: {'learning_rate': 0.7330850647080864, 'max_depth': 18, 'min_child_weight': 2, 'n_estimators

[I 2024-08-13 17:30:38,851] Trial 186 finished with value: -0.8519624867471252 and parameters: {'learning_rate': 0.6807000277272892, 'max_depth': 18, 'min_child_weight': 1, 'n_estimators': 273, 'gamma': 11, 'lambda': 11}. Best is trial 115 with value: -0.2984978041375866.
[I 2024-08-13 17:30:44,384] Trial 187 finished with value: -0.5645496763740939 and parameters: {'learning_rate': 0.6400368416839891, 'max_depth': 20, 'min_child_weight': 7, 'n_estimators': 300, 'gamma': 0, 'lambda': 11}. Best is trial 115 with value: -0.2984978041375866.
[I 2024-08-13 17:30:47,751] Trial 188 finished with value: -0.33685110377687 and parameters: {'learning_rate': 0.7353574681035026, 'max_depth': 18, 'min_child_weight': 2, 'n_estimators': 226, 'gamma': 0, 'lambda': 12}. Best is trial 115 with value: -0.2984978041375866.
[I 2024-08-13 17:30:49,847] Trial 189 finished with value: -0.5066885263958378 and parameters: {'learning_rate': 0.6158012091922518, 'max_depth': 19, 'min_child_weight': 0, 'n_estimator

In [17]:
# Print the best set of hyperparameters
print('Best hyperparameters: ', study.best_params)
# Print the corresponding performance
print('Best performance: ', study.best_value)

Best hyperparameters:  {'learning_rate': 0.5411097064942365, 'max_depth': 16, 'min_child_weight': 2, 'n_estimators': 229, 'gamma': 0, 'lambda': 11}
Best performance:  -0.2984978041375866


In [18]:
set_parameters = study.best_params

In [19]:
def mine_test(X,y, set_parameters= set_parameters): # testa o ajuste
  rows=X.shape[0]
  cols =X.shape[1]

  y_pred   = []

  for i in range(rows):
    X_train = np.delete(X,i,axis=0)
    y_train = np.delete(y,i,axis=0)
    X_test  = X[i,:].reshape(1,cols)
    y_test  = y[i]
    model =  XGBRegressor(**set_parameters)
    model = model.fit(X_train, y_train.reshape(-1,1))
    yi = model.predict(X_test)
    y_pred.append(yi)

  y_pred = np.array(y_pred)
  #print(model)

  return y_pred

In [20]:
y_pred = mine_test(X , y)
y_true = y

In [22]:
print('mae :', mean_absolute_error(y_true, y_pred))
print('rmse :', root_mean_squared_error(y_true, y_pred))
print('r-score :',r2_score(y_true, y_pred))

mae : 0.2984978041375866
rmse : 0.4434466455946895
r-score : 0.8033550725108178


In [23]:
for value in set_parameters:
    print(value,set_parameters[value])

learning_rate 0.5411097064942365
max_depth 16
min_child_weight 2
n_estimators 229
gamma 0
lambda 11
